In [1]:
import socket
import struct
import regex as re
import threading 
import os, sys
import time
import phaseO

MCAST_GRP = '224.0.0.1'
MCAST_PORT = 5050
multiAddrPort = (MCAST_GRP, MCAST_PORT)

IS_MACOS = False
LOCAL_HOST = True

reuse_option = socket.SO_REUSEADDR if not IS_MACOS else socket.SO_REUSEPORT
level_option = socket.IP_ADD_MEMBERSHIP if not LOCAL_HOST else socket.IP_MULTICAST_LOOP

mcast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
mcast_sock.setsockopt(socket.SOL_SOCKET, reuse_option, 1) # for macos, socket.SO_REUSEPORT, linux socket.SO_REUSEADDR

mcast_sock.bind((MCAST_GRP, MCAST_PORT))
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
mcast_sock.setsockopt(socket.IPPROTO_IP, level_option, mreq) # socket.IP_MULTICAST_LOOP for the same machine

uniAddrPort = ("127.0.0.1", 8081)
ucast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
ucast_sock.bind(uniAddrPort)

In [2]:
commands = []

multiRunning = []
multiThread = threading.Thread(target=phaseO.mutiListner, name='multicast', 
    args=(multiRunning, commands, mcast_sock, uniAddrPort))
multiThread.daemon = True
multiThread.start()

uniRunning = []
uniThread = threading.Thread(target=phaseO.uniListner, name='unicast', 
    args=(uniRunning, commands, ucast_sock, uniAddrPort))
uniThread.daemon = True
uniThread.start()

globalSeq = [0]
memberNumber = 1
groupMembers = {memberNumber:uniAddrPort}
FTq = phaseO.FTQueue()
messages = {}
seqNumsSent = {}
testBuildup = []

midRunning = []
midThread = threading.Thread(target=phaseO.middlewareThread, name='middleware', 
    args=(midRunning, globalSeq, memberNumber, groupMembers, 
        FTq, commands, messages, seqNumsSent, 
        ucast_sock, multiAddrPort,[],testBuildup))
midThread.daemon = True
midThread.start()

Running Multicast Listner
Running Unicast Listner


In [3]:
# joining group
joinM = ('joinRequest', memberNumber)
joinM = ",".join(map(lambda x: str(x),joinM))
ucast_sock.sendto(str.encode(joinM), (MCAST_GRP, MCAST_PORT))

Received ['joinConfirm', '0'] from ('127.0.0.1', 8080)


13


Processing command ['joinConfirm', '0'] from ('127.0.0.1', 8080)
Received join confirmation from 0



In [8]:
print("Group Members:\n{}".format(groupMembers))
print("\nFTq.Q:\n{}".format(FTq.Q))
print("\nClient Messages:\n{}".format(messages))
print("\nSequence Numbers:\n{}".format(seqNumsSent))
print("\nGlobal Sequence Num:\n{}".format(globalSeq))

Group Members:
{1: ('127.0.0.1', 8081), 0: ('127.0.0.1', 8080)}

FTq.Q:
{0: {'que': <queue.Queue object at 0x7fe2615c76d0>, 'label': 12}}

Client Messages:
{}

Sequence Numbers:
{1: '86f3c215-f004-40e4-a351-b65376a89634', 3: 'fc96dbf9-f81b-4a89-9133-703b8028ab8c'}

Global Sequence Num:
[4]


In [7]:
testBuildup.append(1)


Re requesting seguence number 2 from group member 0

Received ['seq', 'f44755c4-7757-4804-83d5-b53288200316', '2'] from ('127.0.0.1', 8080)

Processing command ['seq', 'f44755c4-7757-4804-83d5-b53288200316', '2'] from ('127.0.0.1', 8080)
Received sequence number 2 from 0
Sequence number 2 EQUALS 2
Recived queue update command ['qPush', '0', '10']
Incrementing global seq number to 3
Message f44755c4-7757-4804-83d5-b53288200316 processed and un buffered


Caught up with message losses. SQUENCING fc96dbf9-f81b-4a89-9133-703b8028ab8c
Recived queue update command ['qPush', '0', '10']
All caught up. Message f44755c4-7757-4804-83d5-b53288200316 un buffered



In [5]:
FTq.Q[0]['que'].queue

KeyError: 0

Received ['msg', '28ec3a80-c86b-40a0-850a-61b9c115f1cd', 'qCreate', '12'] from ('127.0.0.1', 20001)

Processing command ['msg', '28ec3a80-c86b-40a0-850a-61b9c115f1cd', 'qCreate', '12'] from ('127.0.0.1', 20001)
Received msg ['qCreate', '12'] with id 28ec3a80-c86b-40a0-850a-61b9c115f1cd
NOT SEQUENCING MESSAGE

Received ['seq', '28ec3a80-c86b-40a0-850a-61b9c115f1cd', '0'] from ('127.0.0.1', 8080)

Processing command ['seq', '28ec3a80-c86b-40a0-850a-61b9c115f1cd', '0'] from ('127.0.0.1', 8080)
Received sequence number 0 from 0
Sequence number 0 EQUALS 0
Recived queue update command ['qCreate', '12']
Incrementing global seq number to 1
Message 28ec3a80-c86b-40a0-850a-61b9c115f1cd processed and un buffered

Received ['msg', '86f3c215-f004-40e4-a351-b65376a89634', 'qId', '12'] from ('127.0.0.1', 20001)

Processing command ['msg', '86f3c215-f004-40e4-a351-b65376a89634', 'qId', '12'] from ('127.0.0.1', 20001)
Received msg ['qId', '12'] with id 86f3c215-f004-40e4-a351-b65376a89634
SEQUENCING ME